Import libraries

In [2]:
import pandas as pd
import numpy as np
import re
import time
import seaborn
import requests
import pytrends
from pytrends.request import TrendReq
from bs4 import BeautifulSoup, Comment
from selenium import webdriver
from urllib.parse import urlparse
import warnings

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36',
    'Accept-Language': 'en-US,en;q=0.9',
    'Accept-Encoding': 'gzip, deflate, br',
    'Connection': 'keep-alive',
    'Cache-Control': 'no-cache',
    'Pragma': 'no-cache'
}

In [ ]:
path = r'C:\Users\AshleyNguyen\Desktop\script\\'
df = pd.read_csv(path + 'zombie_companies.csv')

# Display the DataFrame
df

In [95]:
!pip install asyncio
!pip install aiohttp

Preprocess for the async functions

In [ ]:
protocol = 'http://'
df['NewWebsite'] = protocol + df['Website']

In [97]:
test_df = df

In [98]:
# Drop rows with NaN values in 'NewWebsite' column
test_df.dropna(subset=['Website'], inplace=True)

# Reset the index of the DataFrame after dropping rows
test_df.reset_index(drop=True, inplace=True)

In [53]:
test_df = test_df[:15000]

Main async function

In [99]:
import asyncio
import aiohttp
from urllib.parse import urlsplit
import nest_asyncio

async def check_domain_redirect(session, website_url):
    try:
        async with session.get(website_url, headers=headers, timeout=60) as response:
            new_url = str(response.url)
            new_domain = urlsplit(new_url).hostname.lower()
            return new_domain
    except (aiohttp.ClientError, aiohttp.http_exceptions.HttpProcessingError):
        return None
    except asyncio.TimeoutError:
        return None

async def main(websites):
    results = []

    async with aiohttp.ClientSession() as session:
        tasks = []
        for website in websites:
            task = asyncio.create_task(check_domain_redirect(session, website))
            tasks.append(task)

        gathered_results = await asyncio.gather(*tasks)
        for new_domain, website in zip(gathered_results, websites):
            results.append(new_domain)
    return results
    # Store the results in the DataFrame

if __name__ == '__main__':
    # Define the progress bar
    progress_bar = tqdm(range(0, len(test_df['NewWebsite']), 1000))
    lst = []
    
    for i in progress_bar:
        sample = test_df[i:i+1000]
        websites = sample['NewWebsite']
        new_domains = asyncio.run(main(websites))
        lst.append(new_domains)
        # Update the progress bar description
        progress_bar.set_description(f"Processed: {i+1000}/{len(test_df['NewWebsite'])}")

Processed: 26000/33973:  76%|███████▋  | 26/34 [26:26<08:07, 61.00s/it]Can not load response cookies: Illegal key '@cameo/homepageSeed'
Can not load response cookies: Illegal key '@cameo/cameoKidsSession'
Can not load response cookies: Illegal key '@cameo/sessionIdentifier'
Can not load response cookies: Illegal key '@cameo/vid'
Can not load response cookies: Illegal key '@cameo/lang'
Can not load response cookies: Illegal key 'iron-session/snl/next.js'
Processed: 34000/33973: 100%|██████████| 34/34 [34:34<00:00, 61.00s/it]


In [100]:
def flatten_list(nested_list):
    return [item for sublist in nested_list for item in sublist]

flat_list = flatten_list(lst)
print(flat_list)

['appmaster.io', 'www.hugedomains.com', 'www.widbook.com', 'www.hugedomains.com', 'www.modelinia.com', 'cirrus.com', 'findstack.com', 'www.buydomains.com', 'www.buydomains.com', 'www.getblinker.com', None, None, 'www.youmag.com', 'www.tv4ent.com', 'www.visualcv.com', None, 'www.liquidgeneration.com', None, 'showtime.xyz', 'www.brainhunter.com', 'www.collarity.com', 'www.superf3st.xyz', 'inmusik.com', 'www.hugedomains.com', None, 'rayzeapp.com', 'pivot.inc', 'www.hark.com', 'veralia.com', 'www.virtucycling.com', 'dearmedia.com', None, 'www.hugedomains.com', None, None, 'fitmoo.com', 'www.hardcloud.com', 'www.getbetter.com', 'www.hugedomains.com', None, 'www.facebook.com', 'www.ceyx.com', None, 'www.instinctiv.com', None, 'www.sportlobster.com', None, None, 'www.forida.dk', None, None, None, None, 'mazoola.co', 'gobee.bike', None, None, None, 'stageverse.com', 'mobsocmedia.com', 'www.udemy.com', None, None, 'www.miselu.com', None, 'www.assurz.com', 'equip.app', 'www.doolli.com', 'www.myc

In [101]:
test_df['NewDomain'] = flat_list

In [102]:
# Assuming you have a dataframe called 'df' with a column named 'column_name'
value_counts = test_df['NewDomain'].value_counts()

count_none = test_df['NewDomain'].isna().sum()

print(count_none, value_counts)

4658 www.hugedomains.com      232
www.oracle.com            33
www.vmware.com            23
perfectdomain.com         20
www.salesforce.com        16
                        ... 
aclarion.com               1
auction.whois.ai           1
www.oncora.ai              1
bedrockautomation.com      1
goodglamm.com              1
Name: NewDomain, Length: 27911, dtype: int64


In [103]:
merged_df = pd.merge(df, test_df[['CompanyName', 'NewDomain']], on='CompanyName', how='left')

In [108]:
final = pd.merge(df, merged_df[['CompanyName', 'NewDomain_x', 'NewDomain_y']], on='CompanyName', how='left')

In [109]:
final

,prediction,InActive,LastFundingDate,LastFundingNo,TotalFunding,LastFundingAmount,CompanyName,Website,Description,CompanyOwnership,Industry Level 1,Industry Level 2,Industry Level 3,Problem,Note,NewDomain_x,NewDomain_y
0,0.666624,True,11/05/2000 0:00,1,5.000000,5.000000,EasyChem ( Social/Platform Software),NaN,Provides of online services for business entit...,NaN,Information Technology,Software,Social/Platform Software,NaN,NaN,NaN,NaN
1,0.653978,True,27/01/2009 0:00,2,6.000000,6.000000,BrightView Systems,www.brightview-sys.com,Provider of a production technology for photov...,NaN,Energy,Energy Equipment,Alternative Energy Equipment,NaN,NaN,appmaster.io,appmaster.io
2,0.642127,True,18/12/2006 0:00,2,8.500000,2.500000,Industrious Kid,www.industriouskid.com,Operator of a social networking site. The comp...,NaN,Information Technology,Software,Social/Platform Software,NaN,NaN,www.hugedomains.com,www.hugedomains.com
3,0.640157,True,01/01/2013 0:00,1,5.000000,5.000000,Widbook,www.widbook.com,Developer of an online collaborative ebook com...,NaN,Information Technology,Software,Social/Platform Software,NaN,NaN,www.widbook.com,www.widbook.com
4,0.639256,True,10/07/2000 0:00,1,5.830000,5.830000,Cargobiz.com,www.cargobiz.com,Operator of business-to-business e-commerce co...,NaN,Information Technology,Software,Social/Platform Software,NaN,NaN,www.hugedomains.com,www.hugedomains.com
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
35283,0.003425,False,14/07/2022 0:00,11,82.799928,39.999928,Mayvenn,shop.mayvenn.com,Retailer of online hair care products intended...,Privately Held (backing),Consumer Products and Services (B2C),Consumer Non-Durables,Personal Products,NaN,NaN,shop.mayvenn.com,shop.mayvenn.com
35284,0.003409,False,02/02/2022 0:00,11,90.334820,55.515246,Propel (Financial Software),www.joinpropel.com,Developer of a mobile finance platform designe...,Privately Held (backing),Information Technology,Software,Financial Software,NaN,NaN,www.joinpropel.com,www.joinpropel.com
35285,0.003312,False,01/08/2022 0:00,11,377.920116,200.000000,Andela,www.andela.com,Operator of a global talent network intended t...,Privately Held (backing),Business Products and Services (B2B),Commercial Services,Human Capital Services,NaN,NaN,andela.com,andela.com
35286,0.003263,False,28/02/2022 0:00,10,94.591082,50.000003,Qventus,www.qventus.com,Developer of hospital management software desi...,Privately Held (backing),Healthcare,Healthcare Technology Systems,Enterprise Systems (Healthcare),NaN,NaN,qventus.com,qventus.com


In [110]:
final.to_csv(path+'zombie_companies_with_web_redirect.csv', index = False)